In [15]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [16]:
!kaggle datasets download -d kutaykutlu/drowsiness-detection

 95% 171M/180M [00:01<00:00, 124MB/s]
100% 180M/180M [00:01<00:00, 105MB/s]


In [21]:
import tensorflow as tf
from tensorflow.keras.applications import InceptionV3
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dropout,Input,Flatten,Dense,MaxPooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator  # Data Augumentation

In [22]:
import zipfile
zip_ref = zipfile.ZipFile('/content/drowsiness-detection.zip','r')
zip_ref.extractall('/content')
zip_ref.close()

In [24]:

# import os
# import shutil
 
# source = r'/content/archive/train/open_eye'
# destination = r'/content/archive/test/open_eye'
 
# # gather all files
# allfiles = os.listdir(source)
 
# # iterate on all files to move them to destination folder
# count = 1
# for f in allfiles:
#     if count >= 3000:
#         break   
#     src_path = os.path.join(source, f)
#     dst_path = os.path.join(destination, f)
#     shutil.move(src_path, dst_path)
#     count += 1

In [25]:
batchsize=8

In [26]:
train_datagen= ImageDataGenerator(rescale=1./255, rotation_range=0.2,shear_range=0.2,
    zoom_range=0.2,width_shift_range=0.2,
    height_shift_range=0.2, validation_split=0.2)

train_data= train_datagen.flow_from_directory(r'/content/archive/train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical',subset='training' )

validation_data= train_datagen.flow_from_directory(r'/content/archive/train',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical', subset='validation')

Found 33602 images belonging to 2 classes.
Found 8400 images belonging to 2 classes.


In [27]:
test_datagen = ImageDataGenerator(rescale=1./255)

test_data = test_datagen.flow_from_directory(r'/content/archive/test',
                                target_size=(80,80),batch_size=batchsize,class_mode='categorical')

Found 5998 images belonging to 3 classes.


In [28]:
bmodel = InceptionV3(include_top=False, weights='imagenet', input_tensor=Input(shape=(80,80,3)))
hmodel = bmodel.output
hmodel = Flatten()(hmodel)
hmodel = Dense(64, activation='relu')(hmodel)
hmodel = Dropout(0.5)(hmodel)
hmodel = Dense(2,activation= 'softmax')(hmodel)

model = Model(inputs=bmodel.input, outputs= hmodel)
for layer in bmodel.layers:
    layer.trainable = False

In [29]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_2 (InputLayer)           [(None, 80, 80, 3)]  0           []                               
                                                                                                  
 conv2d_94 (Conv2D)             (None, 39, 39, 32)   864         ['input_2[0][0]']                
                                                                                                  
 batch_normalization_94 (BatchN  (None, 39, 39, 32)  96          ['conv2d_94[0][0]']              
 ormalization)                                                                                    
                                                                                                  
 activation_94 (Activation)     (None, 39, 39, 32)   0           ['batch_normalization_94[0]

In [30]:
from tensorflow.keras.callbacks import ModelCheckpoint,EarlyStopping, ReduceLROnPlateau

In [31]:
checkpoint = ModelCheckpoint(r'model.h5',
                            monitor='val_loss',save_best_only=True,verbose=3)

earlystop = EarlyStopping(monitor = 'val_loss', patience=7, verbose= 3, restore_best_weights=True)

learning_rate = ReduceLROnPlateau(monitor= 'val_loss', patience=3, verbose= 3, )

callbacks=[checkpoint,earlystop,learning_rate]

In [32]:
model.compile(optimizer='Adam', loss='categorical_crossentropy',metrics=['accuracy'])

model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,
                   validation_data=validation_data,
                   validation_steps=validation_data.samples//batchsize,
                   callbacks=callbacks,
                    epochs=5)

Epoch 1/5


<ipython-input-32-978403ff37ca>:3: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_data,steps_per_epoch=train_data.samples//batchsize,


4200/4200 [==============================] - ETA: 0s - loss: 0.1813 - accuracy: 0.9279
Epoch 1: val_loss improved from inf to 0.24788, saving model to model.h5
4200/4200 [==============================] - 602s 142ms/step - loss: 0.1813 - accuracy: 0.9279 - val_loss: 0.2479 - val_accuracy: 0.9018 - lr: 0.0010
Epoch 2/5
4200/4200 [==============================] - ETA: 0s - loss: 0.1518 - accuracy: 0.9407
Epoch 2: val_loss did not improve from 0.24788
4200/4200 [==============================] - 598s 142ms/step - loss: 0.1518 - accuracy: 0.9407 - val_loss: 0.2972 - val_accuracy: 0.9076 - lr: 0.0010
Epoch 3/5
4200/4200 [==============================] - ETA: 0s - loss: 0.1364 - accuracy: 0.9470
Epoch 3: val_loss did not improve from 0.24788
4200/4200 [==============================] - 612s 146ms/step - loss: 0.1364 - accuracy: 0.9470 - val_loss: 0.2646 - val_accuracy: 0.9121 - lr: 0.0010
Epoch 4/5
4200/4200 [==============================] - ETA: 0s - loss: 0.1303 - accuracy: 0.9487
Epoch

In [ ]:
# Model Evaluation

In [33]:
acc_tr, loss_tr = model.evaluate_generator(train_data)
print(acc_tr)
print(loss_tr)

<ipython-input-33-87a1e491e135>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_tr, loss_tr = model.evaluate_generator(train_data)


0.10136991739273071
0.9615796804428101


In [13]:
acc_vr, loss_vr = model.evaluate_generator(validation_data)
print(acc_vr)
print(loss_vr)

<ipython-input-13-857f2865b528>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_vr, loss_vr = model.evaluate_generator(validation_data)


0.31895911693573
0.8737541437149048


In [14]:
acc_test, loss_test = model.evaluate_generator(test_data)
print(acc_tr)
print(loss_tr)

<ipython-input-14-229e55840385>:1: UserWarning: `Model.evaluate_generator` is deprecated and will be removed in a future version. Please use `Model.evaluate`, which supports generators.
  acc_test, loss_test = model.evaluate_generator(test_data)


0.04778307303786278
0.9834847450256348
